## Librerias

In [1]:
import numpy as np
import pandas as pd

# stellargraph
import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN

# sklearn
from sklearn import model_selection

#tensorflow
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

import pickle

## Abrimos los datos

In [2]:
with open("./Datos/graphs.txt", "rb") as fp:   # Unpickling ../input/egg-new/26-0.2/graphs.txt
    graphs = pickle.load(fp)

with open("./Datos/labels.txt", "rb") as fp:   # Unpickling
    labels = pickle.load(fp)

# visualizamos como estan los datos importados
print(graphs[0].info())
print(labels[0])

StellarGraph: Undirected multigraph
 Nodes: 25, Edges: 52

 Node types:
  default: [25]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [52]
        Weights: range=[0.659464, 0.961213], mean=0.76453, std=0.0779707
        Features: none
0


## dataframe de los grafos y nodos

In [3]:
summary = pd.DataFrame(
    [(g.number_of_nodes(), g.number_of_edges()) for g in graphs],
    columns=["nodes", "edges"],
)
summary.describe().round(1)

,nodes,edges
count,906.0,906.0
mean,25.0,52.6
std,0.0,35.4
min,25.0,1.0
25%,25.0,25.0
50%,25.0,47.0
75%,25.0,73.0
max,25.0,223.0
